In [3]:
import time
import threading
import random
from collections import deque


def get_vehicle_count(signal):
    
    return random.randint(5, 50)


traffic_lights = {
    'signal_1': 'RED',
    'signal_2': 'RED',
    'signal_3': 'RED',
    'signal_4': 'RED'
}


vehicle_counts = {
    'signal_1': 0,
    'signal_2': 0,
    'signal_3': 0,
    'signal_4': 0
}


recent_green_signals = deque(maxlen=3)

def update_vehicle_counts():
    for signal in vehicle_counts.keys():
        vehicle_counts[signal] = get_vehicle_count(signal)

def open_signal(signal):
    for s in traffic_lights.keys():
        traffic_lights[s] = 'RED'
    traffic_lights[signal] = 'GREEN'
    recent_green_signals.append(signal)
    print(f"Signal {signal} is GREEN")

def change_signal_to_yellow(signal):
    traffic_lights[signal] = 'YELLOW'
    print(f"Signal {signal} is YELLOW")

def display_status():
    while True:
        status = "\n".join([f"{signal}: {state}" for signal, state in traffic_lights.items()])
        print(f"\nTraffic Light Status:\n{status}\n")
        time.sleep(50)  

def manage_traffic():
    while True:
        update_vehicle_counts()
        eligible_signals = [signal for signal in vehicle_counts if signal not in recent_green_signals]
        sorted_signals = sorted(eligible_signals, key=lambda s: vehicle_counts[s], reverse=True)
        
        for signal in sorted_signals:
            open_signal(signal)
            time.sleep(110) 
            change_signal_to_yellow(signal)
            time.sleep(5)  
            break  

def start_traffic_management():
    traffic_management_thread = threading.Thread(target=manage_traffic)
    traffic_management_thread.start()

def start_display_status():
    status_thread = threading.Thread(target=display_status)
    status_thread.start()

if __name__ == "__main__":
    start_traffic_management()
    start_display_status()


Signal signal_4 is GREEN

Traffic Light Status:
signal_1: RED
signal_2: RED
signal_3: RED
signal_4: GREEN



In [3]:
import time
import threading
import random
from collections import deque
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
import os

def get_vehicle_count(signal):
    return random.randint(5, 50)

traffic_lights = {
    'signal_1': 'RED',
    'signal_2': 'RED',
    'signal_3': 'RED',
    'signal_4': 'RED'
}

vehicle_counts = {
    'signal_1': 0,
    'signal_2': 0,
    'signal_3': 0,
    'signal_4': 0
}

recent_green_signals = deque(maxlen=3)


output_directory = r"C:\projects\TMS\Reports"

os.makedirs(output_directory, exist_ok=True)

output_file = os.path.join(output_directory, "traffic_light_status.xlsx")


if not os.path.exists(output_file):
    wb = Workbook()
    ws = wb.active
    ws.title = "Traffic Light Status"
    
    headers = ["Time", "Signal 1 Count", "Signal 2 Count", "Signal 3 Count", "Signal 4 Count", "Active Signal", "Traffic Light State"]
    for col_num, header in enumerate(headers, 1):
        ws[get_column_letter(col_num) + "1"] = header
    wb.save(output_file)

def update_vehicle_counts():
    changed = False
    for signal in vehicle_counts.keys():
        new_count = get_vehicle_count(signal)
        if new_count != vehicle_counts[signal]:
            vehicle_counts[signal] = new_count
            changed = True
    return changed

def open_signal(signal):
    for s in traffic_lights.keys():
        traffic_lights[s] = 'RED'
    traffic_lights[signal] = 'GREEN'
    recent_green_signals.append(signal)
    print(f"Signal {signal} is GREEN")
    save_to_excel(signal, 'GREEN')

def change_signal_to_yellow(signal):
    traffic_lights[signal] = 'YELLOW'
    print(f"Signal {signal} is YELLOW")
    save_to_excel(signal, 'YELLOW')

def save_to_excel(active_signal, state):
    max_retries = 5
    for attempt in range(max_retries):
        try:
            wb = load_workbook(output_file)
            ws = wb.active
            row = [
                time.strftime("%Y-%m-%d %H:%M:%S"),
                vehicle_counts['signal_1'],
                vehicle_counts['signal_2'],
                vehicle_counts['signal_3'],
                vehicle_counts['signal_4'],
                active_signal,
                state
            ]
            ws.append(row)
            wb.save(output_file)
            break
        except PermissionError:
            if attempt < max_retries - 1:
                time.sleep(random.uniform(0.1, 0.5))  
            else:
                print(f"Failed to save data after {max_retries} attempts")

def display_status():
    while True:
        status = "\n".join([f"{signal}: {state}" for signal, state in traffic_lights.items()])
        print(f"\nTraffic Light Status:\n{status}\n")
        time.sleep(50)

def manage_traffic():
    while True:
        if update_vehicle_counts():
            active_signal = next((signal for signal, state in traffic_lights.items() if state != 'RED'), None)
            save_to_excel(active_signal, traffic_lights[active_signal] if active_signal else 'ALL RED')
        
        eligible_signals = [signal for signal in vehicle_counts if signal not in recent_green_signals]
        sorted_signals = sorted(eligible_signals, key=lambda s: vehicle_counts[s], reverse=True)
        for signal in sorted_signals:
            open_signal(signal)
            for _ in range(22):  
                time.sleep(5)
                if update_vehicle_counts():
                    save_to_excel(signal, 'GREEN')
            change_signal_to_yellow(signal)
            time.sleep(5)
            break

def start_traffic_management():
    traffic_management_thread = threading.Thread(target=manage_traffic)
    traffic_management_thread.start()

def start_display_status():
    status_thread = threading.Thread(target=display_status)
    status_thread.start()

if __name__ == "__main__":
    start_traffic_management()
    start_display_status()


Traffic Light Status:
signal_1: RED
signal_2: RED
signal_3: RED
signal_4: RED



Signal signal_3 is GREEN
